Получаем типы сервисов

In [1]:
from townsnet.provision.service_type import ServiceType, Category, AccessibilityType, SupplyType

SERVICE_TYPES = {
    21 : ServiceType(
      id=21, 
      accessibility_value=300, 
      supply_value=61, 
      accessibility_type=AccessibilityType.METERS, 
      supply_type=SupplyType.CAPACITY_PER_1000, 
      category=Category.BASIC,
      weight=0.2
    ),
    63 : ServiceType(
      id=63, 
      accessibility_value=2000,
      supply_value=105.2, 
      accessibility_type=AccessibilityType.METERS, 
      supply_type=SupplyType.CAPACITY_PER_1000, 
      category=Category.ADDITIONAL,
      weight=0.3
    ),
    28 : ServiceType(
      id=28, 
      accessibility_value=10,
      supply_value=13, 
      accessibility_type=AccessibilityType.MINUTES, 
      supply_type=SupplyType.CAPACITY_PER_1000, 
      category=Category.COMFORT,
      weight=0.3
    ),
}

In [2]:
SERVICE_TYPES_NAMES = {
  21: 'kindergarten',
  63: 'cinema',
  28: 'polyclinic',
}

In [3]:
import geopandas as gpd

provisions = {SERVICE_TYPES[st_id] : gpd.read_file(f'./data/res/{st_name}_towns.geojson') for st_id, st_name in SERVICE_TYPES_NAMES.items()}

In [4]:
provisions[SERVICE_TYPES[63]]

,id,name,okrug_name,rayon_name,demand,capacity,capacity_left,demand_left,demand_within,demand_without,provision,geometry
0,0,Болото,Борское сельское поселение,Бокситогорский муниципальный район,1.0,0.0,0.0,0.0,1.0,0.0,1.0,POINT (33.75892 59.36227)
1,1,Большой Остров,Борское сельское поселение,Бокситогорский муниципальный район,2.0,0.0,0.0,0.0,0.0,2.0,0.0,POINT (33.78600 59.47518)
2,2,Бор,Борское сельское поселение,Бокситогорский муниципальный район,2.0,0.0,0.0,0.0,0.0,2.0,0.0,POINT (33.79236 59.47356)
3,3,Бороватое,Борское сельское поселение,Бокситогорский муниципальный район,2.0,0.0,0.0,0.0,0.0,2.0,0.0,POINT (33.77572 59.44249)
4,4,Бочево,Борское сельское поселение,Бокситогорский муниципальный район,1.0,0.0,0.0,0.0,0.0,1.0,0.0,POINT (33.67728 59.32820)
...,...,...,...,...,...,...,...,...,...,...,...,...
2926,2926,Апраксин Бор,Трубникоборское сельское поселение,Тоснненский муниципальный район,1.0,0.0,0.0,0.0,0.0,1.0,0.0,POINT (31.23421 59.17021)
2927,2927,Александровка,Трубникоборское сельское поселение,Тоснненский муниципальный район,1.0,0.0,0.0,0.0,0.0,1.0,0.0,POINT (31.31924 59.18702)
2928,2928,Большая Горка,Трубникоборское сельское поселение,Тоснненский муниципальный район,1.0,0.0,0.0,0.0,0.0,1.0,0.0,POINT (31.47462 59.29408)
2929,2929,Дроздово,Трубникоборское сельское поселение,Тоснненский муниципальный район,1.0,0.0,0.0,0.0,0.0,1.0,0.0,POINT (31.46265 59.30308)


In [5]:
from townsnet.provision.social_model import SocialModel

towns_gdf = list(provisions.values())[0].copy()
sm = SocialModel(towns_gdf, provisions)

In [6]:
project_gdf = gpd.read_file('./data/territories/shlis.geojson')
project_gdf

,building,name,addr:street,addr:housenumber,addr:city,type,is_living,geometry
0,None,None,None,None,None,project,False,"POLYGON ((30.99391 59.92460, 30.99972 59.92939..."


In [8]:
eval = sm.evaluate_provisions(project_gdf.iloc[0].geometry)

In [26]:
import pandas as pd
eval_df = pd.DataFrame([{
  'id' : st.id,
  'weight': st.weight,
  'category': st.category.name,
  'provision': prov
  } for st, prov in eval.items()]).set_index('id', drop=True)
eval_df = eval_df[eval_df.provision >= 0.1]
eval_df

,weight,category,provision
id,,,
28,0.3,COMFORT,0.116279


In [21]:
additional, basic, comfort = eval_df.groupby('category').agg({'weight':'sum'})['weight']